# Wetlands LLM + MCP Testing

Test the LLM with MCP server integration using LangChain

## Quick Start

Before running this notebook, start the local servers:

```bash
./start.sh --local
```

This starts:
- **MCP Server** (port 8001) - MotherDuck MCP server with SSE transport
- **MCP Proxy** (port 8010) - CORS-enabled proxy at `http://localhost:8010/mcp`
- **LLM Proxy** (port 8011) - OpenAI/Anthropic API proxy
- **HTTP Server** (port 8000) - Frontend

In [1]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.tools import tool

## Configuration

In [2]:
# Load config
with open('../maplibre/config.json') as f:
    config = json.load(f)

# Get API key from environment
api_key = os.getenv('NRP_API_KEY')
if not api_key:
    print("WARNING: NRP_API_KEY not set!")
    api_key = input("Enter your NRP API key: ")

# MCP server URL.  Proxy not needed in python which doesn't care about CORS rules
mcp_url = config['mcp_server_url']
llm_endpoint =  config['llm_host'] # config['llm_endpoint']
llm_model = config['llm_model']

print(f"LLM Endpoint: {llm_endpoint}")
print(f"Model: {llm_model}")
print(f"MCP Server: {mcp_url}")

LLM Endpoint: https://ellm.nrp-nautilus.io/v1
Model: glm-v
MCP Server: https://biodiversity-mcp.nrp-nautilus.io/sse


## Load System Prompt

In [3]:
with open('../maplibre/system-prompt.md') as f:
    system_prompt = f.read()

print(f"System prompt loaded: {len(system_prompt)} characters")

System prompt loaded: 6185 characters


## Setup MCP Integration with SSE Transport

Use LangChain's official MCP adapters with SSE (Server-Sent Events) transport:

In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Connect directly to the MCP server's SSE endpoint using config
client = MultiServerMCPClient(
    {
        "wetlands": {
            "transport": "sse",
            "url": mcp_url,
        }
    }
)

# Get tools from the MCP server
tools = await client.get_tools()

print(f"✓ Connected to MCP server!")
print(f"✓ Available tools: {[tool.name for tool in tools]}")

✓ Connected to MCP server!
✓ Available tools: ['query']


## Initialize LLM with MCP Tools

In [5]:
# Initialize LangChain LLM with MCP tools
llm = ChatOpenAI(
    base_url=llm_endpoint,
    api_key=api_key,
    model=llm_model,
    temperature=0.7
)

# Bind the MCP tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print(f"LLM initialized with {len(tools)} MCP tools")

LLM initialized with 1 MCP tools


## Example: Direct Tool Test

Test an MCP tool directly (without the LLM generating the query):

In [6]:
# Find the query tool
query_tool = next((t for t in tools if t.name == "query"), None)

if query_tool:
    # Test it with a simple query
    test_query = """
    CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 's3-west.nrp-nautilus.io', URL_STYLE 'path');
    SELECT COUNT(*) as total_hexagons 
    FROM read_parquet('s3://public-wetlands/hex/**')
    WHERE Z > 0;
    """
    
    result = await query_tool.ainvoke({"query": test_query})
    print("Result:")
    print(result)
else:
    print("Query tool not found!")

Result:
+----------------+
| total_hexagons |
|     BIGINT     |
+----------------+
|   647463329    |
+----------------+


## Example: LLM with MCP Tool Calling

Now let's use the LLM to automatically generate SQL queries and call the MCP tools:

In [7]:
async def ask_wetlands_question(question: str):
    """Ask a natural language question about wetlands data"""
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=question)
    ]
    
    response = await llm_with_tools.ainvoke(messages)
    
    print(f"\n{'='*70}")
    print(f"❓ {question}")
    print(f"{'='*70}\n")
    
    # Handle tool calls
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool = next((t for t in tools if t.name == tool_call['name']), None)
            
            if tool:
                print(f"🔧 Generating and executing SQL query...\n")
                print(tool_call['args']['query'])
                print("-" * 70)
                
                # Execute the tool
                try:
                    result = await tool.ainvoke(tool_call['args'])
                    
                    print(f"\n📊 Raw result:")
                    print(result)
                    print("-" * 70)
                    
                    # Get LLM interpretation
                    messages.append(response)
                    messages.append(HumanMessage(content=f"Tool result: {result}"))
                    final = await llm.ainvoke(messages)
                    
                    print(f"\n💬 Answer:")
                    print(final.content)
                    return final.content
                except Exception as e:
                    print(f"❌ Error: {e}")
                    return None
    else:
        print(f"💬 {response.content}")
        return response.content

## Example Questions

Ask natural language questions - the LLM will generate SQL and use the MCP tools automatically:

In [8]:
# Example 1: Total wetlands area
await ask_wetlands_question("How many hectares of wetlands are there in total?")


❓ How many hectares of wetlands are there in total?

🔧 Generating and executing SQL query...

SET THREADS=100; INSTALL httpfs; LOAD httpfs; CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 's3-west.nrp-nautilus.io', URL_STYLE 'path'); SELECT COUNT(DISTINCT h8) * 73.7327598 as total_hectares FROM read_parquet('s3://public-wetlands/hex/**') WHERE Z > 0;
----------------------------------------------------------------------
❌ Error: Connection closed
❌ Error: Connection closed


In [9]:
# Example 2: Peatlands
await ask_wetlands_question("What is the total area of peatlands in square kilometers?")


❓ What is the total area of peatlands in square kilometers?

🔧 Generating and executing SQL query...

SET THREADS=100; INSTALL httpfs; LOAD httpfs; CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 's3-west.nrp-nautilus.io', URL_STYLE 'path'); SELECT COUNT(DISTINCT h8) * 0.737327598 as total_km2 FROM read_parquet('s3://public-wetlands/hex/**') WHERE Z BETWEEN 24 AND 31;
----------------------------------------------------------------------

📊 Raw result:
+--------------------+
|     total_km2      |
|   DECIMAL(29,9)    |
+--------------------+
| 13891246.785026814 |
+--------------------+
----------------------------------------------------------------------

📊 Raw result:
+--------------------+
|     total_km2      |
|   DECIMAL(29,9)    |
+--------------------+
| 13891246.785026814 |
+--------------------+
----------------------------------------------------------------------

💬 Answer:

Based on the analysis of the global wetlands dataset, the total area of peatlands worldwide is app

"\nBased on the analysis of the global wetlands dataset, the total area of peatlands worldwide is approximately **13,891,247 square kilometers**.\n\nThis represents a significant portion of the Earth's wetland ecosystems, with peatlands (which include bogs, fens, mires, and other peat-forming wetlands) covering nearly 14 million square kilometers globally.\n\nTo put this in perspective:\n- This is roughly equivalent to the size of **Africa** (about 30.4 million km²), meaning peatlands cover nearly half of Africa's land area\n- Peatlands represent one of the largest carbon stores on Earth, containing an estimated 500-600 billion tons of carbon\n- These ecosystems are crucial for biodiversity, water regulation, and climate stability\n\nThe calculation was based on counting all H3 hexagons (resolution 8) classified as peatlands (codes 24-31) and converting them to square kilometers using the standard H3 area conversion factor."

In [10]:
# Example 3: Top wetland types
await ask_wetlands_question("What are the top 5 most common wetland types by area?")


❓ What are the top 5 most common wetland types by area?

🔧 Generating and executing SQL query...



KeyError: 'query'

In [ ]:
# Example 4: Your custom question
await ask_wetlands_question("Compare the area of freshwater wetlands to saline wetlands")